# Mantenimiento
- Buscar cosas descripciones de trabajos efectuados por MEL en mantenimiento
- Buscar detenciones

In [2]:
### CORRERLO INICIALENTE ###
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

import pandas as pd
import polars as pl
import dagster as dg
from kdags.resources.dplyr import *
from kdags.resources.tidyr import *
from io import BytesIO
import pdfplumber

from datetime import datetime
import re
import os
from kdags.definitions import *
from kdags.readr import *

context = dg.build_asset_context()
### IGNORAR ###

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Historial de órdenes de trabajo



In [4]:
# CORRRER
work_order_history_df = Readr.Maintenance.read_work_order_history()

## Busqueda específica

In [6]:
# INICIO MODIFICACION
start_date = "2023-06-06"
end_date = "2024-06-15"
equipment_name = "TK859"
# FIN DE MODIFICACION
df = work_order_history_df.copy()
df = (
    df.loc[
        (df["equipment_name"] == equipment_name)
        & (df["start_date"] >= start_date)
        & (df["end_date"] <= end_date)
    ]
    .sort_values("end_date", ascending=False)
    .reset_index(drop=True)
)
df[["start_date", "end_date", "description"]]

,start_date,end_date,description
0,2024-06-10,2024-06-10,Fuga hidráulica
1,2024-06-06,2024-06-07,Rep fis chasis sop TKDiesel 210mm
2,2024-06-07,2024-06-07,98D Mec Serv PM Paso 7 TK859
3,2024-06-04,2024-06-04,Equipo no cierra puerta alzavidrio malo
4,2024-05-27,2024-05-27,IS: Engrasar punto apunto STD
...,...,...,...
406,2023-06-20,2023-06-20,Ex bajo nivel aceite motor
407,2023-06-20,2023-06-20,ALARMA ACEITE MOTOR
408,2023-06-14,2023-06-14,TKD859 SMARTCAP Sin sincronización
409,2023-06-11,2023-06-11,70D Mec Serv PM Paso 5 TK859


In [26]:
# BUSCADOR ESPECIAL
i = 1
print(df.iloc[i]["description"])
print(df.iloc[i]["task_text"])

Cambio MT2 daño en bobina
07.06.2024 05:57:19 UTC (ARAYFC)
 Brief description of what occurred (e.g. basic sequence of events and
 impacts, avoiding the use of personnel names) ...
 Equipo presenta falla de propulsión en terreno, se realiza evaluación en
 terreno:
 -Se realizan pruebas con Meger en MT2 entregando un valor de 3,8 M ohms
 y ademas se realiza prueba en ruta de fuga a tierra no observando
 desviaciones.
 -Se evalua codigo activo 202, se configura tarjeta inversora 2 y equipo
 no presenta codigo en selftest.
 -Se procede a realizar una prueba en ruta nuevamente presentando codigo
 253, se evaluan motores de tracción, detectando daño en MT2 sector
 estator por lo cual corresponde cambio de MT2.
 ----------------------------------------------------------------
 Immediate action taken ...
 Cambio de MT2 por daño en estator.
 ----------------------------------------------------------------
 Define workrequirements; including known materials, specialist labour,
 special tools th

In [27]:
print("Descripciones de mantenimiento:")
df["task_text"].to_list()

Descripciones de mantenimiento:


['11.06.2024 21:50:55 UTC (LATISO)\n\n Cambio aceite MT2, 50 hrs post cambio componente.\n\n\n se crea aviso\n\n\n Cbio Aceite MT2, 50 H servicio post cambio de componente.\n Realizar toma de muestra aceite inicial, cambo de aceite, microfiltrado\n y muestra de aceite final.\n Generar cartilla de tapones magneticos y compartir a Moncon.\n\n\n\n\n\n\n 15.06.2024 11:28:47 UTC (CORTSR)\n • Se retiran herramientas en pañol MEL\n • Se drena aceite del motor de tracción derecho\n • Se toman muestras de aceite\n • Se toman fotografías de los tapones magnéticos\n • Se rellena con 90 lts de aceite Omala 680 el motor de tracción queda\n con nivel normal\n • Seinstala micro filtradora realizando procedimiento solicitado\n • Se retira micro filtradora del sector\n • Se da torque al perno drenaje con 1000 ft/lb\n • Se da torque al tapón magnético con 90 ft/lb\n\n KOMATSU SPOT TURNO N1',
 '07.06.2024 05:57:19 UTC (ARAYFC)\n Brief description of what occurred (e.g. basic sequence of events and\n impa